In [11]:
"""# Chemical
MESH -> CHEBI

# Disease
OMIM -> MONDO

# Genes
NCBIGene -> HGNC
NCBIGene -> RGD
NCBIGene -> MGI
NCBIGene -> ZFIN
NCBIGene -> XenBase
NCBIGene -> FB
NCGIGene -> WB
NCBIGene -> dictyBase
NCBIGene -> PomBase
OMIM -> HGNC

# These genes depend on why panther only uses ENSEMBL ids sometimes, maybe these can't be mapped?
ENSEMBL:ENSDARG -> ZFIN
ENSEMBL:ENSGALG -> NCBIGene
ENSEMBL:ENSMUSG -> MGI
ENSEMBL:ENSRNOG -> RGD"""

"# Chemical\nMESH -> CHEBI\n\n# Disease\nOMIM -> MONDO\n\n# Genes\nNCBIGene -> HGNC\nNCBIGene -> RGD\nNCBIGene -> MGI\nNCBIGene -> ZFIN\nNCBIGene -> XenBase\nNCBIGene -> FB\nNCGIGene -> WB\nNCBIGene -> dictyBase\nNCBIGene -> PomBase\nOMIM -> HGNC\n\n# These genes depend on why panther only uses ENSEMBL ids sometimes, maybe these can't be mapped?\nENSEMBL:ENSDARG -> ZFIN \nENSEMBL:ENSGALG -> NCBIGene\nENSEMBL:ENSMUSG -> MGI \nENSEMBL:ENSRNOG -> RGD"

In [37]:
import json
import pandas as pd

In [2]:
with open("./gene/mgiSample.BGI.json", "r") as mgi_schema:
    mgi_gene = json.load(mgi_schema)

In [5]:
mgi_gene['data'][0].keys()

dict_keys(['basicGeneticEntity', 'name', 'soTermId', 'symbol'])

MGI:5695867


In [51]:
xref_urls = [
     'https://fms.alliancegenome.org/download/BGI_MGI.json.gz',
     'https://fms.alliancegenome.org/download/BGI_RGD.json.gz',
     'https://fms.alliancegenome.org/download/BGI_ZFIN.json.gz',
     'https://fms.alliancegenome.org/download/BGI_FB.json.gz',
     'https://fms.alliancegenome.org/download/BGI_SGD.json.gz',
     'https://fms.alliancegenome.org/download/BGI_WB.json.gz',
     'https://fms.alliancegenome.org/download/BGI_XB.json.gz'
]

In [52]:
file_names = [x.replace(".gz","").split("/")[-1] for x in xref_urls]

In [54]:
file_names

['BGI_MGI.json',
 'BGI_RGD.json',
 'BGI_ZFIN.json',
 'BGI_FB.json',
 'BGI_SGD.json',
 'BGI_WB.json',
 'BGI_XB.json']

In [87]:
data = []
for fn in file_names:
    with open(fn, "r") as gene_schema:
        genes = json.load(gene_schema)
        for index, gene in enumerate(genes['data']):
            gene = gene['basicGeneticEntity']
            xref = [x['id'].replace("NCBI_Gene", "NCBIGene").replace("NCBI_GENE", "NCBIGene") for x in gene['crossReferences'] if "NCBI" in x['id']]
            prim_id = gene['primaryId'].replace("DRSC:XB:","XB:")
            if len(xref):
                ncbi_xref = xref[0]
            else:
                next
            data.append({
                "subject_id":ncbi_xref,
                "predicate_id": "skos:exactMatch",
                "object_id": prim_id
            })


In [88]:
gene_maps = pd.DataFrame(data)

In [90]:
gene_maps['object_prefix'] = gene_maps['object_id'].apply(lambda x:x.split(":")[0])

In [93]:
gene_maps['object_prefix'].value_counts()

MGI     78326
RGD     70623
WB      48773
XB      38760
ZFIN    37592
FB      30240
SGD      7153
Name: object_prefix, dtype: int64

In [94]:
gene_maps

,subject_id,predicate_id,object_id,object_prefix
0,NCBIGene:12631,skos:exactMatch,MGI:101757,MGI
1,NCBIGene:18630,skos:exactMatch,MGI:101758,MGI
2,NCBIGene:20983,skos:exactMatch,MGI:101759,MGI
3,NCBIGene:231769,skos:exactMatch,MGI:101760,MGI
4,NCBIGene:15364,skos:exactMatch,MGI:101761,MGI
...,...,...,...,...
311462,NCBIGene:100498607,skos:exactMatch,XB:XB-GENE-25874555,XB
311463,NCBIGene:108716024,skos:exactMatch,XB:XB-GENE-25874556,XB
311464,NCBIGene:101733943,skos:exactMatch,XB:XB-GENE-25874657,XB
311465,NCBIGene:108707787,skos:exactMatch,XB:XB-GENE-25874658,XB


In [95]:
import requests

In [96]:
r = requests.get("https://www.genenames.org/cgi-bin/download/custom?col=gd_hgnc_id&col=gd_pub_eg_id&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit")

In [103]:
pd.DataFrame([x.split("\t") for x in r.text.split("\n")])

,0,1
0,HGNC ID,NCBI Gene ID
1,HGNC:5,1
2,HGNC:37133,503538
3,HGNC:24086,29974
4,HGNC:6,
...,...,...
48300,HGNC:13200,7791
48301,HGNC:51695,
48302,HGNC:29027,23140
48303,HGNC:24523,26009
